In [6]:
# Create docker container for agent environment
import docker
import os 
import dotenv

dotenv.load_dotenv()

# Create a Docker client from the environment
client = docker.from_env()

# Get GH token for authorization
gh_token = os.getenv("GH_TOKEN")
if not gh_token:
    raise Exception("GH_TOKEN is not set")

# Pull the image (Ubuntu and Git pre-installed)
# Use `ubuntu` base and install git during container creation
container = client.containers.run(
    image="ubuntu:latest",              # Base image
    command="sleep infinity",           # Keep container running
    detach=True,
    tty=True,                           # Enables command execution like a shell
    stdin_open=True
)

# Install git inside the container
exit_code, output = container.exec_run("bash -c 'apt-get update && apt-get install -y git'")
if exit_code != 0:
    _, stderr = output
    container.stop()
    container.remove()
    raise Exception(f"Failed to install git: {stderr.decode()}")

print(f"Successfully installed git on container: {container.id}")

# Configure git
# Write credentials
exit_code, output = container.exec_run("bash -c 'echo \"https://${GITHUB_TOKEN}@github.com\" > ~/.git-credentials'", environment={"GITHUB_TOKEN": gh_token})
if exit_code != 0:
    _, stderr = output
    container.stop()
    container.remove()
    raise Exception(f"Failed to write credentials: {stderr.decode()}")

# Store credentials
exit_code, output = container.exec_run("git config --global credential.helper store")
if exit_code != 0:
    _, stderr = output
    container.stop()
    container.remove()
    raise Exception(f"Failed to tell Git to use stored credentials: {stderr.decode()}")

print(f"Successfully configured git on container: {container.id}")


Successfully configured git on container: 0cb734bbc4586b342f2f34432c9e7518e977fc0fd7f327695aad959a2ec7affb


In [1]:
# Define tools
BASH_CMD = "bash -c"
GIT_CMD = "git"

def parse_command(cmd: str) -> bool:
    allowed_unix = ["ls", "cd", "pwd", "cat", "mkdir"]
    allowed_git = ["add", "commit", "push", "pull", "status"]
    allowed_commands = allowed_unix + allowed_git
    
    if cmd.split()[0] not in allowed_commands:
        return False
    
    return True

def execute_command(cmd: str) -> str:
    exit_code, output = container.exec_run(cmd)
    if exit_code != 0:
        _, stderr = output
        return f"Error: {stderr.decode()}"
    return output.decode()

def execute_unix_command(cmd: str) -> str:
    """
    Executes a Unix shell command inside the Docker container using bash.

    Args:
        cmd (str): The Unix command to execute (e.g., 'ls', 'pwd', 'mkdir test').

    Returns:
        str: The output of the command, or an error message if the command fails.

    Examples:
        >>> execute_unix_command('ls')
        'file1\nfile2\n'
        >>> execute_unix_command('pwd')
        '/root\n'
    """
    return execute_command(f"{BASH_CMD} {cmd}")

def execute_git_command(cmd: str) -> str:
    """
    Executes a Git command inside the Docker container using bash and git.

    Args:
        cmd (str): The Git command to execute (e.g., 'status', 'add .', 'commit -m \"msg\"').

    Returns:
        str: The output of the command, or an error message if the command fails.

    Examples:
        >>> execute_git_command('status')
        'On branch master\\nnothing to commit, working tree clean\\n'
        >>> execute_git_command('add .')
        ''
    """
    return execute_command(f"{BASH_CMD} {GIT_CMD} {cmd}")

def call_tool(tool: str, args: dict) -> str:
    if tool == "execute_unix_command":
        return execute_unix_command(args["cmd"])
    elif tool == "execute_git_command":
        return execute_git_command(args["cmd"])
    else:
        return f"Error: Invalid tool: {tool}"


In [ ]:
# Define agent
from openai import OpenAI
from pydantic import BaseModel
import instructor


system_prompt = """
You are a git agent. You will be given tasks to complete git-based operations like creating a new local repository and pushing it to a remote repository.

You have access to two tools:
1. execute_unix_command(cmd: str) -> str: This tool is for executing unix commands (e.g. ls, cd, mkdir, etc.)
2. execute_git_command(cmd: str) -> str: This tool is for executing git commands (e.g. git init, git add, git commit, git push, etc.)

You may call one tool call per turn, for up to 10 turns before giving you final answer.

In each turn you should respond in the following format:

<think>
[your thoughts here]
</think>
<tool>
JSON with the following fields:
- tool: The name of the tool to call
- args: The arguments to pass to the tool
</tool>

When you are done, give the final answer in the following format:

<answer>
[your final answer here]
</answer>
"""

class ToolCall(BaseModel):
    tool: str
    args: dict

class AgentResponse(BaseModel):
    think: str
    tool: ToolCall
    answer: str


oai = OpenAI()
oai_instructor = instructor.from_openai(oai)

